In [2]:
import numpy as np
from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
mat_data = loadmat('ATT_Face_DB_3D---39.mat')
print(mat_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'ATT_Face_DB_3D', 'NumFilesForEachPerson', 'NumPersons'])


In [8]:
face_DB_3D = mat_data['ATT_Face_DB_3D']
print(face_DB_3D.shape)
face_DB_3D = face_DB_3D[::2, ::2, :]
face_DB_3D = face_DB_3D.T
print(face_DB_3D.shape)

(112, 92, 390)
(390, 46, 56)


In [9]:
data_set_face = np.zeros((390, (46*56)))
for n in range(390):
    dimension = 0
    for i in range(46):
        for j in range(56):
            data_set_face[n, dimension] = face_DB_3D[n, i, j]
            dimension += 1

print(data_set_face.shape)

(390, 2576)


In [11]:
X = []
for i in range(39):
    down_bound = i*10
    upper_bound = (i+1)*10
    X.append(data_set_face[down_bound:upper_bound])

X = np.array(X)

# test data size: 30%  // train data size
X_test = np.zeros((39, 3, 2576))
X_train = np.zeros((39, 7, 2576))

for k in range(39):
    random_indices = np.random.permutation(10)
    X[k] = X[k, random_indices]
    X_test[k] = X[k, 0:3]
    X_train[k] = X[k, 3:10]
    
X = np.transpose(X, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))
X_train = np.transpose(X_train, (0, 2, 1))
print("X: ", X.shape)
print("X_test shape: ", X_test.shape)
print("X_train shape: ", X_train.shape)

X:  (39, 2576, 10)
X_test shape:  (39, 2576, 3)
X_train shape:  (39, 2576, 7)


In [38]:
class Linear_Regression_Classifier:
    def __init__(self, X, λ=0):
        self.X = X
        self.λ = λ
        self.n_classes = X.shape[0]
        self.d = X.shape[1]
        self.coeff_inverses = []
        
        # compute (X^T X + λI)^-1 X^T for each class
        for k in range(self.n_classes):
            XtX = X[k].T @ X[k]
            inv_term = np.linalg.inv(XtX + λ*np.identity(X[k].shape[1]))
            proj_op = inv_term @ X[k].T 
            self.coeff_inverses.append((X[k], proj_op))
    
    def predict(self, Xtest):
        n_test = Xtest.shape[1]
        distances = np.zeros((self.n_classes, n_test))
        
        for k in range(self.n_classes):
            X_k, proj_op = self.coeff_inverses[k]
            alpha = proj_op @ Xtest          # (n_k x n_test)
            x_hat = X_k @ alpha                # (d x n_test)
            residuals = Xtest - x_hat
            distances[k] = np.linalg.norm(residuals, axis=0)
        
        preds = np.argmin(distances, axis=0)
        return preds


In [39]:
LRC = Linear_Regression_Classifier(X_train, λ=0)

accuracy_sum = 0
for i in range(39):
    y_pred = LRC.predict(X_test[i])
    for j in range(len(y_pred)):
        if y_pred[j] == i:
            accuracy_sum += 1

acc = accuracy_sum / (39*3)
print(acc)

0.9658119658119658


In [ ]:
LRC = Linear_Regression_Classifier(X_train, λ=1)

accuracy_sum = 0
for i in range(39):
    y_pred = LRC.predict(X_test[i])
    for j in range(len(y_pred)):
        if y_pred[j] == i:
            accuracy_sum += 1

acc = accuracy_sum / (39*3)
print(acc)

0.9658119658119658
